In [ ]:
# -*- coding: utf-8 -*-
"""model.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/17AKfgqt1Wl11FUp8WJVl3NQP_edA6LPS
"""

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set up constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'
print("Contents of data_dir:")
print(os.listdir(data_dir))

# Verify the folder structure
class_names = ['MEL', 'NV', 'BCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")



Contents of data_dir:
['NV', 'MEL', 'BCC']


In [ ]:
# Set up data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_names
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_names
)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



Found 6683 images belonging to 3 classes.
Found 1669 images belonging to 3 classes.


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Define F1 Score metric as a class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Compile the model with additional metrics
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), F1Score()])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weight_dict = dict(enumerate(class_weights))

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight_dict
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


208/208 ━━━━━━━━━━━━━━━━━━━━ 408s 2s/step - accuracy: 0.5654 - f1_score: 0.3434 - loss: 0.9791 - precision_7: 0.6478 - recall_6: 0.2404 - val_accuracy: 0.7614 - val_f1_score: 0.7450 - val_loss: 0.6901 - val_precision_7: 0.8504 - val_recall_6: 0.6629
Epoch 2/10
  1/208 ━━━━━━━━━━━━━━━━━━━━ 25s 124ms/step - accuracy: 0.7500 - f1_score: 0.7586 - loss: 1.0454 - precision_7: 0.8462 - recall_6: 0.6875

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


208/208 ━━━━━━━━━━━━━━━━━━━━ 31s 150ms/step - accuracy: 0.7500 - f1_score: 0.7586 - loss: 1.0454 - precision_7: 0.8462 - recall_6: 0.6875 - val_accuracy: 1.0000 - val_f1_score: 0.7500 - val_loss: 0.5648 - val_precision_7: 1.0000 - val_recall_6: 0.6000
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 186s 857ms/step - accuracy: 0.6773 - f1_score: 0.6468 - loss: 0.8471 - precision_7: 0.7573 - recall_6: 0.5646 - val_accuracy: 0.7656 - val_f1_score: 0.7563 - val_loss: 0.6425 - val_precision_7: 0.8247 - val_recall_6: 0.6983
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8438 - f1_score: 0.8136 - loss: 0.5901 - precision_7: 0.8889 - recall_6: 0.7500 - val_accuracy: 0.8000 - val_f1_score: 0.6667 - val_loss: 0.5773 - val_precision_7: 0.7500 - val_recall_6: 0.6000
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 179s 843ms/step - accuracy: 0.6861 - f1_score: 0.6741 - loss: 0.7732 - precision_7: 0.7416 - recall_6: 0.6178 - val_accuracy: 0.7085 - val_f1_score: 0.6970 - val_loss: 0.7704 - val_pr

In [ ]:

# Save the model
model.save('/content/drive/My Drive/models/skin_lesion_classifier.h5')

# Print class indices
print("Class indices:", train_generator.class_indices)

# Function to predict image
def predict_image(img_path, nv_threshold=0.7):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale the image

    # Make prediction
    prediction = model.predict(img_array)

    # Apply threshold for NV class
    if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
        # If NV is predicted but below threshold, choose the next highest probability
        predicted_class = class_names[np.argsort(prediction[0])[-2]]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    confidence = np.max(prediction)

    return predicted_class, confidence

Class indices: {'MEL': 0, 'NV': 1, 'BCC': 2}


In [ ]:
# Directory containing test images
finish_dir = '/content/drive/My Drive/finish'

# Interactive prediction loop
while True:
    # Ask user for input
    user_input = input("Enter an image number (1-1000) or 'q' to quit: ")

    if user_input.lower() == 'q':
        break

    try:
        image_number = int(user_input)

        # Find the image file
        for filename in os.listdir(finish_dir):
            if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(finish_dir, filename)

                # Classify the image
                predicted_class, confidence = predict_image(img_path)

                print(f"Image: {filename}")
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")
                print()
                break
        else:
            print(f"No image found with number {image_number}")

    except ValueError:
        print("Invalid input. Please enter a number or 'q' to quit.")

print("Thank you for using the classifier!")

Enter an image number (1-1000) or 'q' to quit: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: 1.jpg
Predicted class: BCC
Confidence: 0.60

Enter an image number (1-1000) or 'q' to quit: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Image: 1.jpg
Predicted class: BCC
Confidence: 0.60

Enter an image number (1-1000) or 'q' to quit: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Image: 2.jpg
Predicted class: MEL
Confidence: 0.56

Enter an image number (1-1000) or 'q' to quit: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Image: 3.jpg
Predicted class: BCC
Confidence: 0.62

Enter an image number (1-1000) or 'q' to quit: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Image: 4.jpg
Predicted class: BCC
Confidence: 0.63

Enter an image number (1-1000) or 'q' to quit: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Image: 5.jpg
Predicted class: MEL
Confidence: 0.56

Enter an image number (1-1000) or 'q' to quit: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Image: 6.jpg
Predicted class: BCC
Confidence: 0.90

Enter an image number (1-1000

KeyboardInterrupt: Interrupted by user

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall

# Define the F1Score class (make sure this matches the class used when saving the model)
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Load the model
model = load_model('/content/drive/My Drive/models/skin_lesion_classifier.h5',
                   custom_objects={'F1Score': F1Score})

# Print the model summary
print("Model Summary:")
model.summary()

# You can also print other information about the model
print("\nModel Inputs:")
print(model.inputs)

print("\nModel Outputs:")
print(model.outputs)

print("\nModel Metrics:")
print(model.metrics_names)

Model Summary:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │         131,3

 Total params: 14,846,789 (56.64 MB)

 Trainable params: 132,099 (516.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)


Model Inputs:
[<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=input_layer_1>]

Model Outputs:
[<KerasTensor shape=(None, 3), dtype=float32, sparse=False, name=keras_tensor_91>]

Model Metrics:
['loss', 'compile_metrics']
